In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase
cred = credentials.Certificate("/home/poiqwepoi/octo_secrets/octogone-2024-fs.json")
firebase_admin.initialize_app(cred)

# Get Firestore client
db = firestore.client()


In [2]:
collections = db.collections()
for collection in collections:
    print(collection.id)



clientSuppliers
clients
distributors
octogone
products


# Show sample of client documents

In [3]:
# Get a reference to the 'clients' collection
clients_ref = db.collection("clients")

# Fetch up to 5 documents (adjust as needed)
docs = clients_ref.limit(5).stream()

# Print each document
for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)  # Separator for readability


Document ID: 0UlMhnf6vyvsqNxRjqqO
{'id': '0UlMhnf6vyvsqNxRjqqO', 'name': 'Marriott Château Champlain'}
----------------------------------------
Document ID: 1CizJe6Mc8lUi3I6ul9n
{'id': '1CizJe6Mc8lUi3I6ul9n', 'name': 'Sushi Taxi'}
----------------------------------------
Document ID: 7DG0oD842HhqOuWLuXV2
{'id': '7DG0oD842HhqOuWLuXV2', 'name': 'Chez Mag Fine Cantine'}
----------------------------------------
Document ID: ADevDb
{'isQrFeatureActive': True, 'id': 'ADevDb', 'isDev': True, 'clientAccess': {'accessDeniedReason': 0, 'isAccessDenied': False}, 'isBeta': True, 'isAttachmentModuleActive': True, 'name': '# DEV Octane'}
----------------------------------------
Document ID: AInventoryDebugDev
{'isBeta': True, 'id': 'AInventoryDebugDev', 'isDev': True, 'name': '# DEV DEBUG GROS INVENTAIRE'}
----------------------------------------


In [4]:
query = db.collection("clients").stream()

for doc in query:
    data = doc.to_dict()
    if "name" in data and "Rioux" in data["name"]:
        print(f"Document ID: {doc.id}")
        print(data)
        print("-" * 40)


Document ID: chezRioux
{'isBeta': True, 'id': 'chezRioux', 'clientAccess': {'accessDeniedReason': 0, 'isAccessDenied': False}, 'isDev': True, 'name': 'Chez Rioux & Pettigrew'}
----------------------------------------


In [5]:
# find in product collection product with document ID 01ed23998d6dccc6acf86ab638f9c5c1
product = db.collection('clients').document('chezRioux').get()
print(product.to_dict())

{'isBeta': True, 'id': 'chezRioux', 'clientAccess': {'accessDeniedReason': 0, 'isAccessDenied': False}, 'isDev': True, 'name': 'Chez Rioux & Pettigrew'}


In [6]:
doc_ref = db.collection("clients").document("chezRioux")

# List subcollections
subcollections = doc_ref.collections()

for subcollection in subcollections:
    print(subcollection.id)  # Print subcollection names



establishments


In [7]:
subcollection_ref = db.collection("clients").document("chezRioux").collection("establishments")
docs = subcollection_ref.stream()

for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)


Document ID: 6Qw8d1MPRJIKcQslZgeN
{}
----------------------------------------
Document ID: Ili92LFZK5lIfpiqr4QG
{}
----------------------------------------
Document ID: Osxbu9tbdpkoZsjzyyDS
{}
----------------------------------------
Document ID: T5mYEr675p60onvHJbSt
{}
----------------------------------------
Document ID: c40RbOkXOSuSxn9vlG16
{}
----------------------------------------
Document ID: global
{}
----------------------------------------
Document ID: r94BSJnNSarzeE05SxOg
{}
----------------------------------------


In [8]:
global_doc_ref = db.collection("clients").document("chezRioux").collection("establishments").document("global")

# List subcollections
subcollections = global_doc_ref.collections()

for subcollection in subcollections:
    print(subcollection.id)  # Print subcollection names



categories
establishments
products
recipes
suppliers
tags


In [19]:
import json
import pandas as pd

products_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("products")
)

docs = products_ref.stream()


for doc in docs:
    print(f"Document ID: {doc.id}")
    print(doc.to_dict())  # Convert Firestore document to dictionary
    print("-" * 40)

data =[]

for doc in docs:
    myprod = doc.to_dict()

    # Extract required fields
    product_id = doc.id
    product_name = myprod.get("name", None)
    text_search = myprod.get("textSearch", None)
    product_suppliers = myprod.get("productSuppliers", None)
    
    # Extract latestInvoiceLine details
    latest_invoice = myprod.get("latestInvoiceLine", {})

    if latest_invoice:
        # Extract pricePerUnit directly
        price_per_unit = latest_invoice.get("pricePerUnit", None)

        # Extract packaging and price from supplierInvoiceLineInfo
        supplier_info = latest_invoice.get("supplierInvoiceLineInfo", "{}")
        supplier_info = json.loads(supplier_info) if isinstance(supplier_info, str) else supplier_info

        packaging = supplier_info.get("packaging", None)
        price = supplier_info.get("price", None)
    else:
        price_per_unit = None
        packaging = None
        price = None

    # Append structured data to list
    data.append({
        "id": product_id,
        "name": product_name,
        "textSearch": text_search,
        "productSuppliers": product_suppliers,
        "packaging": packaging,
        "price": price,
        "pricePerUnit": price_per_unit,
    })

# Convert to DataFrame
products_df = pd.DataFrame(data)

# Display the DataFrame
print(products_df)



                                    id  \
0     00061f6e22f35ce790a93ef01fa14047   
1     005da4fb77dbbc4b63edbdd1635f8654   
2                 01063MxQwtVWQj0esmob   
3     0201855a8452be6e0bea9e805ddee867   
4     020547e94e6798d7138ee14f23d9885c   
...                                ...   
3105              zxiWefJ0dJfbZHXtVOwO   
3106              zxpejLpqRncFqmZawNEY   
3107              zy2v3m1hyy0CswgarLXO   
3108              zzhM52ZE07HhvZU8DUCo   
3109              zzuEK7hhGx3zUE5ZZ3Jq   

                                              name  \
0                                Fromage St-Honoré   
1                             épice Mustard Powder   
2                          pain burger sans gluten   
3     Assaisonnement ail rôti et poivron une étape   
4                        Purée de fraises, congelé   
...                                            ...   
3105                    Jus de pomme portion 300ml   
3106       De Negri Monovitigno Grappa di prosecco   
3107     

In [22]:
print(products_df[["id", "name", "packaging","pricePerUnit"]])
products_df.to_csv("products.csv", index=False)

                                    id  \
0     00061f6e22f35ce790a93ef01fa14047   
1     005da4fb77dbbc4b63edbdd1635f8654   
2                 01063MxQwtVWQj0esmob   
3     0201855a8452be6e0bea9e805ddee867   
4     020547e94e6798d7138ee14f23d9885c   
...                                ...   
3105              zxiWefJ0dJfbZHXtVOwO   
3106              zxpejLpqRncFqmZawNEY   
3107              zy2v3m1hyy0CswgarLXO   
3108              zzhM52ZE07HhvZU8DUCo   
3109              zzuEK7hhGx3zUE5ZZ3Jq   

                                              name packaging  pricePerUnit  
0                                Fromage St-Honoré      None           NaN  
1                             épice Mustard Powder      None           NaN  
2                          pain burger sans gluten      None           NaN  
3     Assaisonnement ail rôti et poivron une étape   12X660G     19.560000  
4                        Purée de fraises, congelé      None           NaN  
...                              

In [ ]:
"""myprod = doc.to_dict()

myprod.keys()
#print(myprod.productSuppliers, " ", myprod.name, " " , myprod.textSearch)


# print values for keys productSuppliers, name and textSearch
print(myprod["productSuppliers"])
print(myprod["name"])
print(myprod["textSearch"])
print(myprod["id"])
print(myprod["pricePerUnit"])
"""


{'nl335PEacrsOYpqR8HRl2040021': {'equivalences': [{'isActive': True, 'createdId': '010bbb7f485743c3a0d810b568b9', 'measureType': 0, 'equivalence': {'measureId': '4mwgSAFKoNSRfDIZykoL', 'qtyOfMeasure': 8}, 'abbreviation': 'CS', 'createdDate': 1704382052630, 'name': 'CS'}], 'supplierId': 'nl335PEacrsOYpqR8HRl', 'priorityIndex': 2, 'id': 'g0Z8qE2PjSVTNvjELmdC', 'code': '2040021'}, 'nl335PEacrsOYpqR8HRl2040020': {'equivalences': [{'isActive': True, 'createdId': '010bbb7f485743c3a0d810b568b9', 'measureType': 0, 'equivalence': {'measureId': '4mwgSAFKoNSRfDIZykoL', 'qtyOfMeasure': 42}, 'abbreviation': 'CS', 'createdDate': 1704382052630, 'name': 'CS'}], 'supplierId': 'nl335PEacrsOYpqR8HRl', 'priorityIndex': 1, 'id': 'DZJBN5c7sxyEBn5Ary9i', 'code': '2040020'}, 'UeRNZf0p9FHnx54VTu5j700973': {'equivalences': [{'isActive': True, 'createdId': 'ApiInvoice', 'measureType': 0, 'equivalence': {'measureId': '4mwgSAFKoNSRfDIZykoL', 'qtyOfMeasure': 12}, 'abbreviation': 'UN', 'createdDate': 1700744199551, 

KeyError: 'pricePerUnit'

In [12]:
import pandas as pd

# Reference to products collection
products_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("products")
)

# Retrieve documents
docs = products_ref.stream()

# Store data in a list
data = []

for doc in docs:
    myprod = doc.to_dict()
    
    # Ensure required fields exist to avoid KeyError
    data.append({
        "id": doc.id,
        "productSuppliers": myprod.get("productSuppliers", None),
        "name": myprod.get("name", None),
        "textSearch": myprod.get("textSearch", None),
    })

# Convert to DataFrame
prod_df = pd.DataFrame(data)

# Display DataFrame
print(prod_df.head())


                                 id  \
0  00061f6e22f35ce790a93ef01fa14047   
1  005da4fb77dbbc4b63edbdd1635f8654   
2              01063MxQwtVWQj0esmob   
3  0201855a8452be6e0bea9e805ddee867   
4  020547e94e6798d7138ee14f23d9885c   

                                    productSuppliers  \
0  {'6O1FtPg5V9woBCwOqtY241150': {'equivalences':...   
1  {'nl335PEacrsOYpqR8HRl2370012': {'equivalences...   
2                                                 {}   
3  {'XzKzAD6uPnLz7JLKBFqn8787005': {'equivalences...   
4  {'nl335PEacrsOYpqR8HRl940401': {'equivalences'...   

                                           name  \
0                             Fromage St-Honoré   
1                          épice Mustard Powder   
2                       pain burger sans gluten   
3  Assaisonnement ail rôti et poivron une étape   
4                     Purée de fraises, congelé   

                                          textSearch  
0                           Fromage St-Honoré  41150  
1          

In [14]:
df_product_name = pd.read_csv('octogone_products_cleaned_dedup_final.csv')

In [15]:
df_product_name.head()

,product_id,product_name,translated_product,cleaned_product
0,0000a650e53d21b0611e52b2ed519d5d,Pansa Blanca Alella Espagne,White Raspberry Alella Spain,WHITE RASPBERRY ALELLA SPAIN
1,00061f6e22f35ce790a93ef01fa14047,Fromage StHonoré,StHonoré cheese,STHONOR CHEESE
2,000667be92381e018f850053c2ae5ab4,Huile dolive extra vierge,Extra virgin olive oil,EXTRA VIRGIN OLIVE OIL
3,000a8eb819da914d234272f72baef1e4,Peau de poulet,Chicken skin,CHICKEN SKIN
4,0014a71dbbddd58a71a897ae8bb0404d,Oignon Rouge Extra Large,Extra Large Red Onion,EXTRA LARGE RED ONION


In [16]:
# Merge prod_df with df_product_name on id = product_id
merged_df = prod_df.merge(df_product_name, left_on="id", right_on="product_id", how="left")

# Display merged DataFrame
print(merged_df)


                                    id  \
0     00061f6e22f35ce790a93ef01fa14047   
1     005da4fb77dbbc4b63edbdd1635f8654   
2                 01063MxQwtVWQj0esmob   
3     0201855a8452be6e0bea9e805ddee867   
4     020547e94e6798d7138ee14f23d9885c   
...                                ...   
3118              zxiWefJ0dJfbZHXtVOwO   
3119              zxpejLpqRncFqmZawNEY   
3120              zy2v3m1hyy0CswgarLXO   
3121              zzhM52ZE07HhvZU8DUCo   
3122              zzuEK7hhGx3zUE5ZZ3Jq   

                                       productSuppliers  \
0     {'6O1FtPg5V9woBCwOqtY241150': {'equivalences':...   
1     {'nl335PEacrsOYpqR8HRl2370012': {'equivalences...   
2                                                    {}   
3     {'XzKzAD6uPnLz7JLKBFqn8787005': {'equivalences...   
4     {'nl335PEacrsOYpqR8HRl940401': {'equivalences'...   
...                                                 ...   
3118  {'XzKzAD6uPnLz7JLKBFqn9530026': {'equivalences...   
3119  {'2YbkFlGIasSqv7f

In [17]:
pd.set_option("display.max_colwidth", None)

# Show first 5 non-null productSuppliers values
print(merged_df["productSuppliers"].dropna().head(5))



0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            {'6O1FtPg5V9woBCwOqtY241150': {'equivalences': [{'isActive': True, 'id': 'ee3e8bb515be11ee9eadfdcd97790229', 'createdId': '08df2ccbc81444b7aaf5ad9a2f71', 'measureType': 1, 'equivalence': {'measureId': 'RtSe4oh5jnIeUSdNH74A', 'qtyOfMeasure': 1}, 'abbreviation': 'KG', 'textSearch': 'kg', 'createdDate': 1688062464507, 'name': 'KG'}]

In [18]:
filtered_df = merged_df[merged_df["product_id"] == "zzuEK7hhGx3zUE5ZZ3Jq"]

# Display the row
print(filtered_df[["id", "name", "product_name", "productSuppliers"]])




                        id        name product_name  \
3122  zzuEK7hhGx3zUE5ZZ3Jq  Kiwi Frais   Kiwi Frais   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [19]:
selected_df = filtered_df[["id", "name", "product_name", "productSuppliers"]]

# Display the new DataFrame
print(selected_df.iloc[0])


id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [20]:
# Filter rows where productSuppliers contains productSupplier.id = '1fU2P79RXOWhxkCmKax6'
filtered_df = merged_df[
    merged_df["productSuppliers"].apply(
        lambda suppliers: isinstance(suppliers, list) and any(
            supplier.get("id") == "1fU2P79RXOWhxkCmKax6" for supplier in suppliers
        )
    )
]

# Display the filtered DataFrame
print(filtered_df)

Empty DataFrame
Columns: [id, productSuppliers, name, textSearch, product_id, product_name, translated_product, cleaned_product]
Index: []


In [21]:
supplier_data = []

for index, suppliers in merged_df["productSuppliers"].dropna().items():
    if isinstance(suppliers, dict):  # Ensure it's a dictionary
        for supplier_key, supplier_info in suppliers.items():
            supplier_data.append({
                "product_id": merged_df.loc[index, "id"],  # Original product ID
                "supplier_id": supplier_info.get("supplierId", None),
                "product_code": supplier_info.get("code", None),
                "priorityIndex": supplier_info.get("priorityIndex", None),
                "equivalences": supplier_info.get("equivalences", []),
            })

# Convert to DataFrame
supplier_df = pd.DataFrame(supplier_data)

# Display DataFrame
print(supplier_df)


                            product_id           supplier_id product_code  \
0     00061f6e22f35ce790a93ef01fa14047  6O1FtPg5V9woBCwOqtY2        41150   
1     005da4fb77dbbc4b63edbdd1635f8654  nl335PEacrsOYpqR8HRl      2370012   
2     0201855a8452be6e0bea9e805ddee867  XzKzAD6uPnLz7JLKBFqn      8787005   
3     020547e94e6798d7138ee14f23d9885c  nl335PEacrsOYpqR8HRl       940401   
4     020547e94e6798d7138ee14f23d9885c  XzKzAD6uPnLz7JLKBFqn      1186288   
...                                ...                   ...          ...   
3953              zy2v3m1hyy0CswgarLXO  2YbkFlGIasSqv7fjssPs     11724979   
3954              zzhM52ZE07HhvZU8DUCo  XzKzAD6uPnLz7JLKBFqn      1145418   
3955              zzuEK7hhGx3zUE5ZZ3Jq  nl335PEacrsOYpqR8HRl      2040021   
3956              zzuEK7hhGx3zUE5ZZ3Jq  nl335PEacrsOYpqR8HRl      2040020   
3957              zzuEK7hhGx3zUE5ZZ3Jq  UeRNZf0p9FHnx54VTu5j       700973   

      priorityIndex  \
0               0.0   
1               0.0   
2     

In [22]:
equivalence_data = []

for _, row in supplier_df.iterrows():
    for eq in row["equivalences"]:
        equivalence_data.append({
            "product_id": row["product_id"],
            "supplier_id": row["supplier_id"],
            "product_code": row["product_code"],cleaned_products.ipynb
            "measure_id": eq.get("equivalence", {}).get("measureId", None),
            "qty_of_measure": eq.get("equivalence", {}).get("qtyOfMeasure", None),
            "abbreviation": eq.get("abbreviation", None),
        })

# Convert to DataFrame
equivalence_df = pd.DataFrame(equivalence_data)

# Display DataFrame
print(equivalence_df)


                            product_id           supplier_id product_code  \
0     00061f6e22f35ce790a93ef01fa14047  6O1FtPg5V9woBCwOqtY2        41150   
1     005da4fb77dbbc4b63edbdd1635f8654  nl335PEacrsOYpqR8HRl      2370012   
2     0201855a8452be6e0bea9e805ddee867  XzKzAD6uPnLz7JLKBFqn      8787005   
3     0201855a8452be6e0bea9e805ddee867  XzKzAD6uPnLz7JLKBFqn      8787005   
4     020547e94e6798d7138ee14f23d9885c  nl335PEacrsOYpqR8HRl       940401   
...                                ...                   ...          ...   
4061              zzhM52ZE07HhvZU8DUCo  XzKzAD6uPnLz7JLKBFqn      1145418   
4062              zzhM52ZE07HhvZU8DUCo  XzKzAD6uPnLz7JLKBFqn      1145418   
4063              zzuEK7hhGx3zUE5ZZ3Jq  nl335PEacrsOYpqR8HRl      2040021   
4064              zzuEK7hhGx3zUE5ZZ3Jq  nl335PEacrsOYpqR8HRl      2040020   
4065              zzuEK7hhGx3zUE5ZZ3Jq  UeRNZf0p9FHnx54VTu5j       700973   

                measure_id  qty_of_measure abbreviation  
0     RtSe4oh5jnI

In [23]:
import pandas as pd

# Step 1: Extract supplier-level data with product name
supplier_data = []

for index, suppliers in merged_df["productSuppliers"].dropna().items():
    if isinstance(suppliers, dict):  # Ensure it's a dictionary
        for supplier_key, supplier_info in suppliers.items():
            supplier_data.append({
                "product_id": merged_df.loc[index, "id"],  # Original product ID
                "name": merged_df.loc[index, "name"],  # Product Name
                "supplier_id": supplier_info.get("supplierId", None),
                "product_code": supplier_info.get("code", None),
                "priorityIndex": supplier_info.get("priorityIndex", None),
                "equivalences": supplier_info.get("equivalences", []),
            })

supplier_df = pd.DataFrame(supplier_data)

# Step 2: Extract equivalence-level data
equivalence_data = []

for _, row in supplier_df.iterrows():
    for eq in row["equivalences"]:
        equivalence_data.append({
            "product_id": row["product_id"],
            "name": row["name"],  # Keep product name
            "supplier_id": row["supplier_id"],
            "product_code": row["product_code"],
            "measure_id": eq.get("equivalence", {}).get("measureId", None),
            "qty_of_measure": eq.get("equivalence", {}).get("qtyOfMeasure", None),
            "abbreviation": eq.get("abbreviation", None),
        })

equivalence_df = pd.DataFrame(equivalence_data)

# Step 3: Merge supplier_df and equivalence_df
final_df = supplier_df.drop(columns=["equivalences"]).merge(equivalence_df, on=["product_id", "name", "supplier_id", "product_code"], how="left")

# Display final DataFrame
print(final_df)


                            product_id  \
0     00061f6e22f35ce790a93ef01fa14047   
1     005da4fb77dbbc4b63edbdd1635f8654   
2     0201855a8452be6e0bea9e805ddee867   
3     0201855a8452be6e0bea9e805ddee867   
4     020547e94e6798d7138ee14f23d9885c   
...                                ...   
4809              zzhM52ZE07HhvZU8DUCo   
4810              zzhM52ZE07HhvZU8DUCo   
4811              zzuEK7hhGx3zUE5ZZ3Jq   
4812              zzuEK7hhGx3zUE5ZZ3Jq   
4813              zzuEK7hhGx3zUE5ZZ3Jq   

                                              name           supplier_id  \
0                                Fromage St-Honoré  6O1FtPg5V9woBCwOqtY2   
1                             épice Mustard Powder  nl335PEacrsOYpqR8HRl   
2     Assaisonnement ail rôti et poivron une étape  XzKzAD6uPnLz7JLKBFqn   
3     Assaisonnement ail rôti et poivron une étape  XzKzAD6uPnLz7JLKBFqn   
4                        Purée de fraises, congelé  nl335PEacrsOYpqR8HRl   
...                                    

In [63]:
final_df.to_csv('final_df.csv', index=False)

In [24]:
# Reference to suppliers collection
suppliers_ref = (
    db.collection("clients")
    .document("chezRioux")
    .collection("establishments")
    .document("global")
    .collection("suppliers")
)

# Fetch suppliers
supplier_docs = suppliers_ref.stream()

# Store supplier data in a list
supplier_data = []

for doc in supplier_docs:
    supplier_info = doc.to_dict()
    supplier_data.append({
        "supplier_id": doc.id,  # Firestore document ID
        "supplier_name": supplier_info.get("name", None),
        "supplier_email": supplier_info.get("email", None),
        "supplier_website": supplier_info.get("website", None),
    })

# Convert supplier data to DataFrame
supplier_df = pd.DataFrame(supplier_data)

# Display the supplier DataFrame
print(supplier_df)




              supplier_id                                     supplier_name  \
0    0JPaDkdZH6hgujFNJSxP                                   Jac le chévrier   
1    1J1XBpih7Q8qizuedlIl                                   Vente du Dragon   
2    1NyBnJ01NQ22uBlpqgRl                         Lavage de vitres National   
3    1WiKS8nniLKUv1Is0JMe                      Heritage Food Service Gr Can   
4    1fU2P79RXOWhxkCmKax6                                    Maison JoyHill   
..                    ...                                               ...   
395  zNaZYFe3xadoPZ5HMRWu  Pierre cloutier.  proprietaire. alain reparateur   
396  zYSV4NtorWdezKcuQx4i                     Fleury et Associés - Notaires   
397  zhkvaVEjTLOZPdc5DS2s                                          Alambika   
398  zlHH6BsWf4nubov4pfS8                                    Morille Québec   
399  znUz8Jkxx9gSp69hvOcl                       La Villa vinaigre et jardin   

    supplier_email                            suppl

In [25]:
# Merge final_df with supplier_df on 'supplier_id'
final_df = final_df.merge(supplier_df, on="supplier_id", how="left")

# Display the updated DataFrame
print(final_df)


                            product_id  \
0     00061f6e22f35ce790a93ef01fa14047   
1     005da4fb77dbbc4b63edbdd1635f8654   
2     0201855a8452be6e0bea9e805ddee867   
3     0201855a8452be6e0bea9e805ddee867   
4     020547e94e6798d7138ee14f23d9885c   
...                                ...   
4809              zzhM52ZE07HhvZU8DUCo   
4810              zzhM52ZE07HhvZU8DUCo   
4811              zzuEK7hhGx3zUE5ZZ3Jq   
4812              zzuEK7hhGx3zUE5ZZ3Jq   
4813              zzuEK7hhGx3zUE5ZZ3Jq   

                                              name           supplier_id  \
0                                Fromage St-Honoré  6O1FtPg5V9woBCwOqtY2   
1                             épice Mustard Powder  nl335PEacrsOYpqR8HRl   
2     Assaisonnement ail rôti et poivron une étape  XzKzAD6uPnLz7JLKBFqn   
3     Assaisonnement ail rôti et poivron une étape  XzKzAD6uPnLz7JLKBFqn   
4                        Purée de fraises, congelé  nl335PEacrsOYpqR8HRl   
...                                    

In [27]:
# Export final_df to an Excel file
final_df.to_excel("final_products_suppliers.xlsx", index=False)

print("Export complete: final_products_suppliers.xlsx")


Export complete: final_products_suppliers.xlsx
